# Dublin City Area analysis


## Summary

### The problem

Many new businesses fail to make profit within their first three years of opening (59% in the hospitality industry, according to a study by Dr. HG Parsa).

A particularly difficult area of business is the restaurant/food industry. This industry is oversaturated and highly competitive. For this reason, new business owners hoping to open a resaurant, in an already saturated market, should aim to find a location for opening their resaurant that gives them the best possible chance for success.    

### Proposed solution 

This project will analyse the postal code areas of Dublin, Ireland. Data about the given areas will be collected using the Foursquare API. 
Geographical information will be retrieved from the web using the python library BeautifulSoup.  

The data from Foursquare will be used to cluster the areas into groups based on the popularity of localised amenities. This data will then be used to 
make recommendations to business owners about the location of optimal area for setting up a new business.

#### Foursquare sample response
Bellow is a sample JSON response from the Foursquare API. In this project we will be using the latitude and longitude coordinates of restaurants across Dublin city in order to get an idea of how saurated the restaurant market is within each postal code. We can then use this information to look at areas of lower saturation as possinble locations for a new business.


```json
{
  "meta": {
    "code": 200,
    "requestId": "5ac51d7e6a607143d811cecb"
  },
  "response": {
    "venues": [
      {
        "id": "5642aef9498e51025cf4a7a5",
        "name": "Mr. Purple",
        "location": {
          "address": "180 Orchard St",
          "crossStreet": "btwn Houston & Stanton St",
          "lat": 40.72173744277209,
          "lng": -73.98800687282996,
          "labeledLatLngs": [
            {
              "label": "display",
              "lat": 40.72173744277209,
              "lng": -73.98800687282996
            }
          ],
          "distance": 8,
          "postalCode": "10002",
          "cc": "US",
          "city": "New York",
          "state": "NY",
          "country": "United States",
          "formattedAddress": [
            "180 Orchard St (btwn Houston & Stanton St)",
            "New York, NY 10002",
            "United States"
          ]
        },
        "categories": [
          {
            "id": "4bf58dd8d48988d1d5941735",
            "name": "Hotel Bar",
            "pluralName": "Hotel Bars",
            "shortName": "Hotel Bar",
            "icon": {
              "prefix": "https://ss3.4sqi.net/img/categories_v2/travel/hotel_bar_",
              "suffix": ".png"
            },
            "primary": true
          }
        ],
        "venuePage": {
          "id": "150747252"
        }
      }
    ]
  }
}
```